In [1]:
import os
import sys
import pathlib
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))

import tqdm
import numpy as np
import pickle
import matplotlib.cm as cm
import matplotlib.axes as am
import matplotlib.pyplot as plt
%matplotlib widget

import dimod
import dwave
import dwave.system
from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
import dwave.inspector
import dwave_networkx as dnx
import minorminer

from src.particle_funcs import distance_matrix as distance_matrix
from src.particle_funcs import io as particles_io
import src.leap_funcs.qubo.q_matrix as q_matrix

from src import leap_funcs as leap_funcs
from src.leap_funcs import embedding_quality
from src.leap_funcs.qubo import parameterstudy

from src import h5py_funcs
from src.h5py_funcs import inspections, discoveries, init_custom_getstates, io, parameterstudy_using_info_file

no cupy available, imported numpy as usual
Custom getstate functions for dwave.cloud.config.models.ClientConfig, dwave.cloud.client.qpu.Client, dwave.cloud.solver.StructuredSolver, dwave.system.samplers.dwave_sampler.DWaveSampler, dwave.system.composites.embedding.FixedEmbeddingComposite have been initialized.


In [2]:
import io
io.DEFAULT_BUFFER_SIZE

8192

In [4]:
study_name = 'sub_2'
info_file_name = r'study_params_small.h5'
study_name = 'sub_3'
study_name = 'sub_4_2'
#study_name = 'sub_5'
#study_name = 'sub_5_1_1'
#study_name = 'sub_5_2'
#study_name = 'sub_5_3'
#study_name = 'sub_5_3_2'
study_name = 'sub_7_1_2'

#study_subs = ['', '_2']
study_subs = ['']

info_name = r'study_params_sub7_1_2'

info_file_name = [info_name + sub + '.h5' for sub in study_subs]


study_folder_path = [pathlib.Path.cwd().joinpath('01_out',study_name+sub) for sub in study_subs]

info_file_name_path = [pathlib.Path.joinpath(sfp, ifn) for sfp, ifn in zip(study_folder_path, info_file_name)]
samples_folder_name = r'samples'
samples_folder_name_path = [pathlib.Path.joinpath(sfp, samples_folder_name) for sfp in study_folder_path]
print(str(info_file_name_path[0])[-3:])
assert np.array([i.exists() for i in info_file_name_path]).all(), f'Info file does not exist. Check info_file_name_path ({info_file_name_path})'
info_file_name_path

.h5


[PosixPath('/mnt/userdrive/Users/home/adam-1aeqn8vhvpjnv4u/git_repos/Quantum_Annealing_for_Particle_Matching/00_tests/01_out/sub_7_1_2/study_params_sub7_1_2.h5')]

In [5]:
dict_infos_read = {i:h5py_funcs.inspections.read_info_file_to_dict(info_file_name_path=ifp, infoset_name = 'info')  for i,ifp in enumerate(info_file_name_path)}
#dddddd = dict_infos_read.copy()

dict_info_read = {}
#for key in reversed(dict_infos_read.keys()): #reversed to make sure info from initial study_file is contained as a|=b priorizes b
#    dict_info_read = dict_info_read | dddddd[key]

for i,ifp in enumerate(info_file_name_path):
    dict_info_read.update(h5py_funcs.inspections.read_info_file_to_dict(info_file_name_path=ifp, infoset_name = 'info'))

#dict_info_read


#orig:
#dict_info_read = h5py_funcs.inspections.read_info_file_to_dict(info_file_name_path=info_file_name_path, infoset_name = 'info')
#dict_info_read.keys()

##########
#
# The following test fails, because comparing dicts throws an error (below) when arrays are encountered, thats why it is left to the user to make sure the subs are compatible with the main study.
# "The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()" 
#
#########
#if len(info_file_name_path)>1:
#    for i in range(len(info_file_name_path)):
#        for key, value in dict_infos_read[0].items():
#            if key=='time_history': continue
#            else:
#                print(key)
#                if isinstance(value, np.ndarray):
#                    assert (value == dict_infos_read[i][key]).all(), print(value, dict_infos_read[i][key])
#                else:
#                    try: value == dict_infos_read[i][key]
#                    except Exception as e:
#                        print(e)
#                        print(value)
#                        print(dict_infos_read[i][key])

In [6]:
#print(dict_infos_read[0]['time_history'])
for i in range(len(dict_infos_read)):
    _tmp = []
    for key, value in dict_infos_read[i]['time_history'].items():
        if key == 'attrs':
            continue
        else:
            if not value['attrs']['finished']:
                _tmp.append(key)
    print(len(_tmp), _tmp)

_tmp = []
for key, value in dict_info_read['time_history'].items():
    if key == 'attrs':
        continue
    else:
        if not value['attrs']['finished']:
            _tmp.append(key)
print(len(_tmp), _tmp)
del _tmp

1087 ['zz_7105867748', 'zz_6294749264', 'zz_1182152534', 'zz_3788333593', 'zz_6252757989', 'zz_3711002418', 'zz_5170232572', 'zz_0173913551', 'zz_8286069094', 'zz_5121445422', 'zz_1250807590', 'zz_1191386760', 'zz_8232979284', 'zz_8269785493', 'zz_1394326228', 'zz_6987779257', 'zz_2978168333', 'zz_3674909592', 'zz_5408998207', 'zz_7030838980', 'zz_8735178788', 'zz_5641196207', 'zz_5163406618', 'zz_2156425748', 'zz_7851602705', 'zz_1402665339', 'zz_8015011822', 'zz_4991575769', 'zz_6296219828', 'zz_6806658382', 'zz_8488320056', 'zz_3593503585', 'zz_1734146968', 'zz_3959076513', 'zz_7304951386', 'zz_6528997792', 'zz_4550721825', 'zz_5191913298', 'zz_7168207997', 'zz_6117789188', 'zz_7168806810', 'zz_1263163945', 'zz_7622249105', 'zz_2200207428', 'zz_3305549993', 'zz_1282128333', 'zz_8399076939', 'zz_4279068470', 'zz_7005719685', 'zz_8886172753', 'zz_6272628070', 'zz_9931588090', 'zz_5479239864', 'zz_0343447865', 'zz_2381405344', 'zz_3770330101', 'zz_7415047758', 'zz_7340717507', 'zz_2648

In [7]:
list_studies = list(dict_info_read['time_history'].keys())
list_studies.remove('attrs')
print(len(list_studies), list_studies)

2816 ['zz_8097006340', 'zz_3271526003', 'zz_5771100733', 'zz_6926620307', 'zz_0035448496', 'zz_5187077316', 'zz_0283319908', 'zz_7078527454', 'zz_1607154550', 'zz_0718801486', 'zz_4880416337', 'zz_3199431448', 'zz_6141606936', 'zz_3686594361', 'zz_3751853524', 'zz_7074557852', 'zz_4896149692', 'zz_6859867594', 'zz_6431177729', 'zz_3692460835', 'zz_9344025413', 'zz_8669389062', 'zz_1063149707', 'zz_3365147581', 'zz_2666922764', 'zz_7772712132', 'zz_2728652188', 'zz_3246246724', 'zz_8085842554', 'zz_4199855208', 'zz_2188248880', 'zz_3078711298', 'zz_9052476602', 'zz_9127208387', 'zz_4198252939', 'zz_2169158755', 'zz_0335574155', 'zz_3626572964', 'zz_8480470983', 'zz_6816191017', 'zz_9801759557', 'zz_8144652514', 'zz_2948642163', 'zz_2314053580', 'zz_9045097699', 'zz_4138906829', 'zz_4607778612', 'zz_1900500435', 'zz_4989572363', 'zz_7252953223', 'zz_8371351367', 'zz_8058220185', 'zz_3049613848', 'zz_0731748568', 'zz_6339681232', 'zz_1154740184', 'zz_6380713589', 'zz_8202971060', 'zz_6917

In [8]:
#print(dict_info_read['time_history']['zz_7293366434']['data'][2].decode('utf-8')[:4] == '2024')
#print(dict_info_read['time_history']['zz_7293366434']['attrs']['finished'])

In [9]:
for stud in list_studies:
    if (not dict_info_read['time_history'][stud]['attrs']['finished'])\
        and (dict_info_read['time_history'][stud]['data'][2].decode('utf-8')[:4] == '2024'):
        print('here 1', stud)
    elif dict_info_read['time_history'][stud]['attrs']['finished']\
        and (dict_info_read['time_history'][stud]['data'][2].decode('utf-8')[:4] != '2024'):
        print('here 1_2', stud)
    elif dict_info_read['time_history'][stud]['attrs']['finished']\
        and (dict_info_read['time_history'][stud]['data'][2].decode('utf-8')[:4] == '2024'):
        #print('here 2')
        continue
    else:
        print('here else', stud)
        for key, value in dict_infos_read.items():
            if '2024' == value['time_history'][stud]['data'][2].decode('utf-8')[:4]:
                dict_info_read['time_history'][stud]['data'][1] = (value['time_history'][stud]['data'][1]+'.'.encode('utf-8'))[:-1]
                dict_info_read['time_history'][stud]['data'][2] = (value['time_history'][stud]['data'][2]+'.'.encode('utf-8'))[:-1]
                dict_info_read['time_history'][stud]['attrs']['finished'] = True


here else zz_7105867748
here else zz_6294749264
here else zz_1182152534
here else zz_3788333593
here else zz_6252757989
here else zz_3711002418
here else zz_5170232572
here else zz_0173913551
here else zz_8286069094
here else zz_5121445422
here else zz_1250807590
here else zz_1191386760
here else zz_8232979284
here else zz_8269785493
here else zz_1394326228
here else zz_6987779257
here else zz_2978168333
here else zz_3674909592
here else zz_5408998207
here else zz_7030838980
here else zz_8735178788
here else zz_5641196207
here else zz_5163406618
here else zz_2156425748
here else zz_7851602705
here else zz_1402665339
here else zz_8015011822
here else zz_4991575769
here else zz_6296219828
here else zz_6806658382
here else zz_8488320056
here else zz_3593503585
here else zz_1734146968
here else zz_3959076513
here else zz_7304951386
here else zz_6528997792
here else zz_4550721825
here else zz_5191913298
here else zz_7168207997
here else zz_6117789188
here else zz_7168806810
here else zz_126

In [10]:
for stud in list_studies:
    if not dict_info_read['time_history'][stud]['attrs']['finished']:
        print(stud, dict_info_read['time_history'][stud]['data'])

zz_7105867748 [b'2024-11-29T16:25:14+0100' b'2024-11-30T20:33:50+0100'
 b'1910-11-03T00:31:00']
zz_6294749264 [b'2024-11-29T16:25:14+0100' b'2024-11-30T22:58:36+0100'
 b'1910-11-03T00:31:00']
zz_1182152534 [b'2024-11-29T16:25:14+0100' b'2024-11-30T22:58:40+0100'
 b'1910-11-03T00:31:00']
zz_3788333593 [b'2024-11-29T16:25:14+0100' b'2024-11-30T22:58:39+0100'
 b'1910-11-03T00:31:00']
zz_6252757989 [b'2024-11-29T16:25:14+0100' b'2024-11-30T22:58:40+0100'
 b'1910-11-03T00:31:00']
zz_3711002418 [b'2024-11-29T16:25:14+0100' b'2024-11-30T22:58:39+0100'
 b'1910-11-03T00:31:00']
zz_5170232572 [b'2024-11-29T16:25:14+0100' b'2024-11-30T22:58:40+0100'
 b'1910-11-03T00:31:00']
zz_0173913551 [b'2024-11-29T16:25:14+0100' b'2024-11-30T22:58:41+0100'
 b'1910-11-03T00:31:00']
zz_8286069094 [b'2024-11-29T16:25:14+0100' b'2024-11-30T22:58:42+0100'
 b'1910-11-03T00:31:00']
zz_5121445422 [b'2024-11-29T16:25:14+0100' b'2024-11-30T22:58:41+0100'
 b'1910-11-03T00:31:00']
zz_1250807590 [b'2024-11-29T16:25:14+010

In [11]:
print(dict_info_read['study'])
study_read = dict_info_read['study']['data']
study_read.dtype.names
est_runts = study_read['sets']['estimated_runtime']
cum_est_runts = np.sum(est_runts)
print(est_runts)
print('cummulative estimated runtime for 1000 samples each is [h]:', cum_est_runts/3600)
del study_read, est_runts, cum_est_runts


{'attrs': {}, 'data': array([((  4.5919823 ,  932.48291174, 2040.61592231, 0.02601395, 3.19761581, 0.18596916,  0.15253127, 0.62547142, 0.,  0.41745294,  0.83490587,  1.25235881,   1.66981174,   2.08726468,   2.50471762,   2.92217055,   3.33962349,   3.75707642,   4.17452936,   4.5919823 , 0., 1.16418444e-01, 2.26917764e-01, 3.31798907e-01, 4.31347523e-01, 5.25834736e-01, 6.15517885e-01, 7.00641227e-01, 7.81436596e-01, 8.58124044e-01, 0.93091243, 1., 2.18012193), b'zz_8097006340'),
       ((  4.6967697 ,  932.48291174, 2040.61592231, 0.02601395, 3.19761581, 0.18596916,  0.15253127, 0.62547142, 0.,  0.42697906,  0.85395813,  1.28093719,   1.70791626,   2.13489532,   2.56187438,   2.98885345,   3.41583251,   3.84281158,   4.26979064,   4.6967697 , 0., 1.17040484e-01, 2.27998013e-01, 3.33188737e-01, 4.32912375e-01, 5.27453068e-01, 6.17080190e-01, 7.02049114e-01, 7.82601942e-01, 8.58968190e-01, 0.93136545, 1., 2.18022672), b'zz_3271526003'),
       ((  4.5919823 ,   91.74887091, 2040.61592

In [12]:
array_identifiers, started_psets, finished_psets = h5py_funcs.inspections.extract_identifiers(dict_info_read = dict_info_read)

print(started_psets)
print(finished_psets)
print('{p1}/{p2} p_sets have been started, \n {p3}/{p2} p_sets have been finished'.format(p1=len(started_psets), p2=array_identifiers.shape[0], p3=len(finished_psets)))

list_array_identifiers = []; list_started_psets = []; list_finished_psets = []
appenders = [list_array_identifiers, list_started_psets, list_finished_psets]
for d in dict_infos_read.values():
    appendees = h5py_funcs.inspections.extract_identifiers(dict_info_read = d)
    print(len(appendees[0]), len(appendees[1]), len(appendees[2]))
    assert (array_identifiers==appendees[0]).all()
    list_array_identifiers.append(appendees[0])
    if len(list_started_psets) > 0:
        print('here if')
        list_started_psets.append([ap for ap in appendees[1] if not ap in list_started_psets[0]])
        list_finished_psets.append([ap for ap in appendees[2] if not ap in list_finished_psets[0]])
    else:
        print('here else')
        list_started_psets.append(appendees[1])
        list_finished_psets.append(appendees[2])

print(list_started_psets)
print(list_finished_psets)
print('{p1}/{p2} p_sets have been started, \n {p3}/{p2} p_sets have been finished'.\
      format(p1=[len(l) for l in list_started_psets], p2=[len(l) for l in list_array_identifiers], p3=[len(l) for l in list_finished_psets]))

[b'zz_8097006340', b'zz_3271526003', b'zz_5771100733', b'zz_6926620307', b'zz_0035448496', b'zz_5187077316', b'zz_0283319908', b'zz_7078527454', b'zz_1607154550', b'zz_0718801486', b'zz_4880416337', b'zz_3199431448', b'zz_6141606936', b'zz_3686594361', b'zz_3751853524', b'zz_7074557852', b'zz_4896149692', b'zz_6859867594', b'zz_6431177729', b'zz_3692460835', b'zz_9344025413', b'zz_8669389062', b'zz_1063149707', b'zz_3365147581', b'zz_2666922764', b'zz_7772712132', b'zz_2728652188', b'zz_3246246724', b'zz_8085842554', b'zz_4199855208', b'zz_2188248880', b'zz_3078711298', b'zz_9052476602', b'zz_9127208387', b'zz_4198252939', b'zz_2169158755', b'zz_0335574155', b'zz_3626572964', b'zz_8480470983', b'zz_6816191017', b'zz_9801759557', b'zz_8144652514', b'zz_2948642163', b'zz_2314053580', b'zz_9045097699', b'zz_4138906829', b'zz_4607778612', b'zz_1900500435', b'zz_4989572363', b'zz_7252953223', b'zz_8371351367', b'zz_8058220185', b'zz_3049613848', b'zz_0731748568', b'zz_6339681232', b'zz_1154

In [ ]:
is_read_from_sample_data = False

In [ ]:
dict_for_df = {}
if is_read_from_sample_data:
    for sfnp, ids in zip(samples_folder_name_path,list_finished_psets):
        print('reading from', sfnp)
        _tmp_study_name = sfnp.parent.stem
        print('file name', _tmp_study_name)
        #print(os.path.exists(sfnp))
        #for id in ids:
        #    _tmp = os.path.join(sfnp, id.decode('utf-8')+'.h5')
        #    print(os.path.exists(_tmp), _tmp)
        dict_for_df |= h5py_funcs.inspections.read_answers_to_dict(\
            samples_folder_name_path=sfnp\
            , array_identifiers=np.array(ids))#[::25]
    print(dict_for_df.__sizeof__()) # size in memory in bytes
    print(len(dict_for_df)) # number of keys in dict
    with open(pathlib.Path('01_out',f'dict_for_df_{_tmp_study_name}.txt'), 'wb') as f:
        pickle.dump(dict_for_df, f, protocol=5)
    del dict_for_df, _tmp_study_name


In [ ]:
if not is_read_from_sample_data:
    with open(pathlib.Path('01_out',f'dict_for_df_{study_name}.txt'), 'rb') as f:
        dict_for_df = pickle.load(f)

In [ ]:
is_combine_pickled_dicts = False
if is_combine_pickled_dicts:
    in_dicts = ['01_out\\dict_for_df_sub_6_2.txt'\
               ,'01_out\\dict_for_df_sub_6_2_2.txt']
    out_dicts = ['01_out\\dict_for_df_sub_6_2_combined.txt']
    out_dict = {}
    for in_d in in_dicts:
        print(f'read {in_d}')
        with open(in_d, 'rb') as f:
            out_dict |= pickle.load(f)
    print('finished reading all in_dicts')
    for key, val in out_dict.items():
        sample_set_length = (len(val['custom']['sampleset'].keys()))
        if sample_set_length != 10:
            print(key, sample_set_length)

In [ ]:
if is_combine_pickled_dicts:
    out_dicts = ['01_out\\dict_for_df_sub_5_3_combined.txt']
    with open(out_dicts[0], 'wb') as f:
        pickle.dump(out_dict, f, protocol=5)

In [ ]:
import ast
import dwave.samplers

num_particles = 5
qubo = dict_info_read['qubos'][f'{num_particles}_{num_particles}']

#sim_annealing_sample = dimod.samplers.ExactSolver().sample_qubo(
#    {ast.literal_eval(key): value['data'] for key, value in qubo.items()})
sim_annealing_sample = dwave.samplers.SimulatedAnnealingSampler().sample_qubo(
    {ast.literal_eval(key): value['data'] for key, value in qubo.items()},
    num_reads=10000)
sim_annealing_sample = sim_annealing_sample.aggregate() # accumulates number of occurences

In [ ]:
type(sim_annealing_sample)

In [ ]:
exact_sol = sim_annealing_sample.record
exact_sol.sort(order='energy')
exact_sol

In [ ]:
n_samples_to_compare = 3
n_exact_sols_to_compare = 3
#colour_label = 'fraction_samples_matched_3_samps_3_sols'
colour_label = 'fraction_samples_is_found_best'
dir_name_path_plots = './01_out/'
is_train_gp = False
training_iterations = 5 # only required if is_train_gp==True

In [ ]:
success_dict = h5py_funcs.inspections.extract_success_dict(\
    dict_for_df = dict_for_df\
    , exact_sols = exact_sol\
    , n_samples_to_compare = n_samples_to_compare\
    , n_exact_sols_to_compare = n_exact_sols_to_compare)


for key in success_dict.keys():
    print(key, success_dict[key])

In [ ]:
#fig = h5py_funcs.inspections.return_bar_plot(\
#    success_dict = success_dict\
#    , n_samples_to_compare = n_samples_to_compare\
#    , n_exact_sols_to_compare = n_exact_sols_to_compare)



In [ ]:
a = np.zeros((dict_info_read['study']['data'].shape[0],))
print(a)
print(dict_info_read['study']['data']['sets'].shape)
print(dict_info_read['study']['data']['sets'].dtype.isalignedstruct)
b = np.lib.recfunctions.rec_append_fields(dict_info_read['study']['data'], 'param 1', a)
print(b)

In [ ]:
print(b.dtype)

In [ ]:
dict_info_read['study']['data']['sets'].copy().view(dtype=np.float64, type=np.ndarray)

In [ ]:
started_sets, study_matched_started_ids = h5py_funcs.inspections.extract_started_sets_from_success_dict(\
    success_dict = success_dict\
    , dict_info_read = dict_info_read)
print(started_sets)
print(study_matched_started_ids)
print(study_matched_started_ids.dtype)

In [ ]:
if -1 != study_name.find('sub_6'):
    _tmp_names_sets = list(study_matched_started_ids['sets'].dtype.names)
    print(_tmp_names_sets)
    for r in ['s00','s11'\
              , 't00', 't01', 't02', 't03', 't04', 't05', 't06', 't07', 't08', 't09', 't10']:
        _tmp_names_sets.remove(r)
    print(_tmp_names_sets)
    
    _tmp_dtype = study_matched_started_ids.dtype
    print(_tmp_dtype)
    _tmp_dtype_2 = _tmp_dtype['sets'][_tmp_names_sets]
    print(_tmp_dtype_2.base.names)
    print(dir(_tmp_dtype_2.base))
    print(dir(_tmp_dtype_2.base.fields))
    print(_tmp_dtype_2.base.fields)
    _tmp_dtype_3 = np.dtype({'names':['sets','identifiers','started sets'], 'formats':[_tmp_dtype_2,'S13','S13']})
    print(_tmp_dtype_3)
    _tmp_dtype_4 = [(key, val[0]) for key, val in _tmp_dtype_2.base.fields.items()]
    print(_tmp_dtype_4)
    _tmp_dtype_5 = np.dtype({'names':['sets','identifiers','started sets'], 'formats':[_tmp_dtype_4,'S13','S13']})
    print(_tmp_dtype_5)
    
    
    print()
    
    _tmp_array_1 = study_matched_started_ids['sets'][_tmp_names_sets]
    _tmp_array_2 = study_matched_started_ids['identifiers']
    _tmp_array_3 = study_matched_started_ids['started sets']
    
    _tmp_array_4 = np.rec.fromarrays([_tmp_array_1, _tmp_array_2, _tmp_array_3], dtype=_tmp_dtype_5)
    _tmp_array_4
    print(_tmp_array_4.dtype)
    
    # results in np.array instead of np.recarray, but dont know why
    _tmp_array_5 = np.zeros(shape=study_matched_started_ids.shape, dtype=_tmp_dtype_5)
    _tmp_array_5['sets'] = study_matched_started_ids['sets'][_tmp_names_sets]
    _tmp_array_5['identifiers'] = study_matched_started_ids['identifiers']
    _tmp_array_5['started sets'] = study_matched_started_ids['started sets']
    
    study_matched_started_ids = _tmp_array_4
    #study_matched_started_ids = _tmp_array_4
    del _tmp_dtype, _tmp_dtype_2, _tmp_dtype_3, _tmp_dtype_4, _tmp_dtype_5
    del _tmp_array_1, _tmp_array_2, _tmp_array_3, _tmp_array_4, _tmp_array_5
    print(study_matched_started_ids.dtype)
    dict_info_read['study']['data'] = study_matched_started_ids

In [ ]:
results_names = list(list(success_dict.values())[0].keys())
to_remove = ['is_found_best_per_run', 'num_subs_per_run', 'num_samples_per_run', 'num_samples_per_sub_per_run', 'num_matched_per_run'\
             , 'num_matched_per_sub_per_run', 'num_samples_matched_per_run', 'num_samples_matched_per_sub_per_run', 'submissions', 'num_samples_is_found_best_per_run']
for s in to_remove:
    results_names.remove(s)
print(results_names)



sampler_array,results_names, ids = h5py_funcs.inspections.generate_sampler_array_for_plots(\
    success_dict = success_dict\
    , results_names = results_names\
    , started_sets = started_sets\
    , study_matched_started_ids = study_matched_started_ids\
    , n_samples_to_compare = n_samples_to_compare\
    , n_exact_sols_to_compare = n_exact_sols_to_compare)

print(sampler_array.shape)
print(results_names)

In [ ]:
max_prob = np.max(sampler_array[:,-2])
print("max_prob:", max_prob)
row_of_max = np.argwhere(sampler_array[:,-2] == max_prob)
print(row_of_max)
print(results_names)
sampler_array[row_of_max,:]

In [ ]:
np.argsort(sampler_array[:,-2])

In [ ]:
for i, id in enumerate(success_dict.keys()):
    if success_dict[id]["is_found_best"]:
        print(i, id , success_dict[id]["num_samples_is_found_best"]/success_dict[id]["num_samples"])

In [ ]:
import torch
import gpytorch

In [ ]:
class MultitaskGPModel(gpytorch.models.ExactGP):
                def __init__(self, train_x, train_y, likelihood):
                    super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
                    self.mean_module = gpytorch.means.MultitaskMean(
                        gpytorch.means.ConstantMean(), num_tasks=sampler_array.shape[1]
                    )
                    self.covar_module = gpytorch.kernels.MultitaskKernel(
                        gpytorch.kernels.RBFKernel(), num_tasks=sampler_array.shape[1]
                    )

                def forward(self, x):
                    mean_x = self.mean_module(x)
                    covar_x = self.covar_module(x)
                    return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

            # Instantiate multi-task likelihood and GP model 
if study_name == 'sub_2':
    train_x = torch.tensor(study_matched_started_ids['sets'].view(np.float64).copy()).to(torch.float)
    test_x = torch.tensor(dict_info_read['study']['data']['sets'].view(np.float64).copy()).to(torch.float)
else:
    _tmp_train_dtype = study_matched_started_ids['sets'].dtype
    _tmp_test_dtype = dict_info_read['study']['data']['sets'].dtype
    _tmp_train_data = study_matched_started_ids['sets'].view((np.float64, len(_tmp_train_dtype.names))).copy()
    _tmp_test_data = dict_info_read['study']['data']['sets'].view((np.float64, len(_tmp_test_dtype.names))).copy()
    train_x = torch.tensor(_tmp_train_data).to(torch.float)
    test_x = torch.tensor(_tmp_test_data).to(torch.float)
    del(_tmp_train_dtype, _tmp_test_dtype, _tmp_train_data, _tmp_test_data)

train_y = torch.tensor(sampler_array).to(torch.float)

num_params_in_gp = train_x.shape[1]
num_params_out_gp = train_y.shape[1]

likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=sampler_array.shape[1])
model = MultitaskGPModel(train_x, train_y, likelihood)

In [ ]:
if not is_train_gp:
    print('is_train_gp =', is_train_gp, ' GP training is skipped')
    mean = None
    pass
else:
    # Switch to training mode
    model.train()
    likelihood.train()

    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    # Train GP using training data
    print('Start training')
    print(training_iterations)
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()
        print('  Finished training iteration %i/%i' % (i + 1, training_iterations), 'loss:', loss.item())
    print('Finished training' + '\n')
    # Switch to evaluation mode, and probe trained GP using testing data
    model.eval()
    likelihood.eval()
    print('Start testing')
    with torch.no_grad():
        observed_model = model(test_x)
        print('  Testing: Finished evaluation')
        observed_pred = likelihood(observed_model)
        print('  Testing: Finished likelihood')
        mean = observed_pred.mean
        lower, upper = observed_pred.confidence_region()
    print('Finished testing' + '\n')

    assert mean.shape[1] == num_params_out_gp

In [ ]:
sampler_array.shape

In [ ]:
import importlib
importlib.reload(h5py_funcs.inspections)

In [ ]:
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14
kwargs_pltrc = {'font_size':SMALL_SIZE\
                ,'axes_titlesize': SMALL_SIZE\
                ,'axes_labelsize': MEDIUM_SIZE\
                ,'xtick_labelsize': SMALL_SIZE\
                ,'ytick_labelsize': SMALL_SIZE\
                ,'legend_fontsize': SMALL_SIZE\
                ,'figure_titlesize': BIGGER_SIZE\
                ,'marker_scatter': '2'}

In [ ]:
print(results_names)
print(mean.shape) if np.array(mean).all() else print('GP was not trained')
print(colour_label)
print(dir_name_path_plots)
print(sampler_array.shape)
print(study_matched_started_ids.dtype)
print(study_matched_started_ids.ndim)
print(len(study_matched_started_ids['sets'].dtype.names))
print(study_matched_started_ids['sets'].ndim)
fig2, fig_pp1 = h5py_funcs.inspections.return_plots(study_name = study_name\
        , study_matched_started_ids = study_matched_started_ids\
        , gp_mean = mean, results_names = results_names, dict_info_read = dict_info_read\
        , sampler_array = sampler_array, colour_label = colour_label\
        , axs_types=['linlin'], dir_name_path_plots = dir_name_path_plots\
        , comb_types=['bi'], kwargs_pltrc=kwargs_pltrc)

In [ ]:
#fig2 = fig2.sort_values(by='fraction_samples_is_found_best')
#fig2
#fig_pp1[1]

In [ ]:
#fig2.iloc[:,0]

In [ ]:
plt.close()

In [ ]:

fig_pp1.show()

In [ ]:
import SALib as sa
import SALib.analyze.sobol

In [ ]:
names = study_matched_started_ids['sets'].dtype.names[:3]
bounds = [[study_matched_started_ids['sets'][name].min(), study_matched_started_ids['sets'][name].max()] for name in names]
salib_problem = {
    'num_vars': 3,
    'names': names,
    'bounds': bounds
}
salib_problem

In [ ]:
ar = sampler_array[:,-2]
sa.analyze.sobol.analyze(salib_problem, np.pad(ar, (0, 512-len(ar)), 'constant', constant_values=(0, ar.mean())), print_to_console=True)

In [ ]:
ar = sampler_array[:,-2]
ar2 = np.pad(ar, (0, 512-len(ar)), 'constant', constant_values=(0, 0))

In [ ]:
sampler_array, names